In [1]:
import os
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse
import re
import numpy as np
from tqdm import tqdm
import time
import pysparnn.cluster_index as ci
no_num_clean_p = re.compile(r'[^\w\s]+|\d+', re.UNICODE)

DOCS = 3213835
document_file = "/home/tfink/data/kodicare/trec-2019-dl/doc_ret/msmarco-docs.head100k.tsv"
queries_file = "/home/tfink/data/kodicare/trec-2019-dl/doc_ret/msmarco-doctrain-queries.tsv"
qrels_file = "/home/tfink/data/kodicare/trec-2019-dl/doc_ret/msmarco-doctrain-qrels.tsv"

# triplet generation process
* preprocess text
* create tf idf vectors
    * dictionary
    * tf-idf model
    * convert to vector? (sparse)
* store vectors in faiss (with document id)
* store vectors somewhere else because sparse
* get positive document (PD) vectors 
* get approximate nearest neighbors of PD to create negatives

In [2]:
def document_text_iterator():
    with open(document_file, "r") as fp:
        for doc_line in tqdm(fp, total=DOCS):
            data = doc_line.strip().split(sep="\t")
            if len(data) != 4:
                #print(len(data), data)
                continue
            doc_id, doc_url, doc_title, doc_text = data
            yield doc_text


def document_iterator():
    with open(document_file, "r") as fp:
        for doc_line in fp:
            data = doc_line.strip().split(sep="\t")
            if len(data) != 4:
                #print(len(data), data)
                continue
            doc_id, doc_url, doc_title, doc_text = data
            yield doc_id, doc_url, doc_title, doc_text


def get_document_ids():
    doc_ids = []
    doc_ids_inv = {}
    with open(document_file, "r") as fp:
        for doc_line in fp:
            data = doc_line.strip().split(sep="\t")
            if len(data) != 4:
                #print(len(data), data)
                continue
            doc_id, doc_url, doc_title, doc_text = data
            doc_ids_inv[doc_id] = len(doc_ids)
            doc_ids.append(doc_id)
    return doc_ids, doc_ids_inv

In [3]:
def read_qrels():
    relevance_judgements = {}
    with open(qrels_file, "r") as fp:
        for qrel_line in fp:
            q_id, _, doc_id, relevance = qrel_line.strip().split()
            relevance = int(relevance)
            if relevance == 1:
                if q_id not in relevance_judgements:
                    relevance_judgements[q_id] = set()
                relevance_judgements[q_id].add(doc_id)
    return relevance_judgements


def read_queries():
    queries = {}
    with open(queries_file, "r") as fp:
        for queries_line in fp:
            q_id, q_text = queries_line.strip().split(sep="\t")
            queries[q_id] = q_text
    return queries

In [4]:
tfidf_vect = TfidfVectorizer(max_df=0.25, min_df=100, norm='l2', dtype=np.float32, stop_words='english')
x = tfidf_vect.fit_transform(document_text_iterator())

  0%|          | 0/3213835 [00:00<?, ?it/s]

  3%|▎         | 100000/3213835 [00:42<21:55, 2367.58it/s]


In [5]:
doc_ids, doc_ids_inv = get_document_ids()
relevance_judgements = read_qrels()

In [6]:
# build the search index!
cp = ci.MultiClusterIndex(x, doc_ids)

/home/tfink/miniconda3/envs/kodicare/lib/python3.8/site-packages/pysparnn-0.4-py3.8.egg/pysparnn/matrix_distance.py:192: RuntimeWarning: divide by zero encountered in divide


In [27]:
top_k = 100
k_cluster = 5

search_features_vec = []
doc_data = []
for q_id, relevant_doc_ids in tqdm(relevance_judgements.items()):
    for pos_doc_id in relevant_doc_ids:
        if pos_doc_id not in doc_ids_inv:
            continue
        relevant_doc_vector = x[doc_ids_inv[pos_doc_id]]
        search_features_vec.append(relevant_doc_vector)
        doc_data.append((q_id, pos_doc_id, relevant_doc_ids))


def batched_search(cp, search_features_vec, batch_size=4096, k=10, k_clusters=2, return_distance=True):
    search_features_vec_size = search_features_vec.shape[0]
    steps = int(np.ceil(search_features_vec_size / batch_size))
    sims_with_idx = []
    for i in tqdm(range(steps)):
        start_idx = i*batch_size
        end_idx = start_idx + batch_size
        sims_with_idx.extend(
            cp.search(search_features_vec[start_idx:end_idx,:], k=k, 
                      k_clusters=k_clusters, return_distance=return_distance)
        )
    return sims_with_idx

print("positives", len(search_features_vec))
search_features_vec = scipy.sparse.vstack(search_features_vec)
t0 = time.time()
sims_with_idx = batched_search(cp, search_features_vec, batch_size=1000, k=top_k, k_clusters=k_cluster, return_distance=True)
t1 = time.time()
print("Took", t1-t0)

  0%|          | 0/367013 [00:00<?, ?it/s]

100%|██████████| 367013/367013 [00:00<00:00, 462454.54it/s]


positives 11532


  0%|          | 0/3 [00:00<?, ?it/s]/home/tfink/miniconda3/envs/kodicare/lib/python3.8/site-packages/pysparnn-0.4-py3.8.egg/pysparnn/matrix_distance.py:192: RuntimeWarning: divide by zero encountered in divide
  magnitude = 1.0 / (a_root_sum_square * self.matrix_root_sum_square)
100%|██████████| 3/3 [02:00<00:00, 40.15s/it]

Took 120.44857835769653


In [35]:
upper_bound=1.0
lower_bound=0.40
triplets = []
for i in range(len(doc_data)):
    # set all known relevant documents to -inf
    q_id, pos_doc_id, relevant_doc_ids = doc_data[i]
    for distance, neg_doc_id in sims_with_idx[i]:
        s = 1-float(distance)
        if neg_doc_id in relevant_doc_ids or s > upper_bound or s < lower_bound:
            continue
        triplets.append(((q_id, pos_doc_id, neg_doc_id), s))

In [36]:
triplets[:10]

[(('145', 'D178709', 'D1772147'), 0.54287),
 (('145', 'D178709', 'D657589'), 0.52431685),
 (('145', 'D178709', 'D3256601'), 0.44943464),
 (('145', 'D178709', 'D3025891'), 0.40145660000000005),
 (('295', 'D2370783', 'D2595990'), 0.5140323600000001),
 (('295', 'D2370783', 'D720413'), 0.4991666),
 (('295', 'D2370783', 'D1492605'), 0.4848367),
 (('295', 'D2370783', 'D1657038'), 0.4622452),
 (('295', 'D2370783', 'D651600'), 0.45918304),
 (('295', 'D2370783', 'D2129596'), 0.44762135000000003)]

In [23]:
def batched_cosine_similarity(X,Y, batch_size=4096, dtype=np.float64):
    X_size = X.shape[0]
    Y_size = Y.shape[0]
    steps = int(np.ceil(Y_size / batch_size))
    sims = np.zeros((X_size, Y_size), dtype=dtype)
    for i in tqdm(range(steps)):
        start_idx = i*batch_size
        end_idx = start_idx + batch_size
        sim = cosine_similarity(X,Y[start_idx:end_idx,:]).astype(dtype=dtype)
        sims[:,start_idx:end_idx] = sim
    return sims


doc_vecs = []
doc_data = []
for q_id, relevant_doc_ids in tqdm(relevance_judgements.items()):
    for pos_doc_id in relevant_doc_ids:
        if pos_doc_id not in doc_ids_inv:
            continue
        relevant_doc_vector = x[doc_ids_inv[pos_doc_id]]
        relevant_doc_idx = [doc_ids_inv[doc_id] for doc_id in relevant_doc_ids if doc_id in doc_ids_inv]
        doc_vecs.append(relevant_doc_vector)
        doc_data.append((q_id, pos_doc_id, relevant_doc_idx))

print("positives", len(doc_vecs))
doc_vecs = scipy.sparse.vstack(doc_vecs)
t0 = time.time()
#sims = cosine_similarity(doc_vecs, x)
sims = batched_cosine_similarity(doc_vecs, x, batch_size=4096, dtype=np.float32)
t1 = time.time()
print("Took", t1-t0)

100%|██████████| 367013/367013 [00:01<00:00, 244872.28it/s]


positives 11532


100%|██████████| 25/25 [00:58<00:00,  2.34s/it]

Took 58.432796478271484


In [24]:
top_k=10
upper_bound=1.0
lower_bound=0.40
triplets = []
for i in range(len(doc_data)):
    # set all known relevant documents to -inf
    q_id, pos_doc_id, relevant_doc_idx = doc_data[i]
    sims[i,relevant_doc_idx] = -np.inf
    # set all documents outside the boundaries to -inf
    sims[i] = np.where(np.logical_and(sims[i] <= upper_bound, sims[i] >= lower_bound), sims[i], -np.inf)
    # get the top k document ids with the highest similarity, filtering out -inf
    top_sims_idx = np.argpartition(sims[i,:], -top_k)[-top_k:]
    sims_with_idx = [(doc_ids[idx], s) for idx, s in zip(top_sims_idx, sims[i,:][top_sims_idx]) 
                     if s != -np.inf]
    sims_with_idx = sorted(sims_with_idx, key=lambda x:x[1], reverse=True)
    for neg_doc_id, s in sims_with_idx:
        triplets.append(((q_id, pos_doc_id, neg_doc_id), s))

In [25]:
triplets[:10]

[(('145', 'D178709', 'D1772147'), 0.54287),
 (('145', 'D178709', 'D657589'), 0.52431685),
 (('145', 'D178709', 'D3256601'), 0.44943464),
 (('145', 'D178709', 'D3025891'), 0.40145656),
 (('295', 'D2370783', 'D2595990'), 0.51403236),
 (('295', 'D2370783', 'D720413'), 0.49916664),
 (('295', 'D2370783', 'D100082'), 0.49583653),
 (('295', 'D2370783', 'D1492605'), 0.4848367),
 (('295', 'D2370783', 'D1657038'), 0.46224523),
 (('295', 'D2370783', 'D626914'), 0.4620225)]

In [ ]:
top_k=10
upper_bound=1.0
lower_bound=0.40
batch_size = 1024

def get_triplets():
    triplets = []

    doc_vecs = []
    doc_data = []
    for q_id, relevant_doc_ids in tqdm(relevance_judgements.items()):
        for pos_doc_id in relevant_doc_ids:
            if pos_doc_id not in doc_ids_inv:
                continue
            relevant_doc_vector = x[doc_ids_inv[pos_doc_id]]
            relevant_doc_idx = [doc_ids_inv[doc_id] for doc_id in relevant_doc_ids if doc_id in doc_ids_inv]
            doc_vecs.append(relevant_doc_vector)
            doc_data.append((q_id, pos_doc_id, relevant_doc_idx))

    print("positives", len(doc_vecs))
    doc_vecs = scipy.sparse.vstack(doc_vecs)
    t0 = time.time()
    sims = cosine_similarity(doc_vecs, x)
    t1 = time.time()
    print("Took", t1-t0)

    for i in range(len(doc_data)):
        # set all known relevant documents to -inf
        q_id, pos_doc_id, relevant_doc_idx = doc_data[i]
        sims[i,relevant_doc_idx] = -np.inf
        # set all documents outside the boundaries to -inf
        sims[i] = np.where(np.logical_and(sims[i] <= upper_bound, sims[i] >= lower_bound), sims[i], -np.inf)
        # get the top k document ids with the highest similarity, filtering out -inf
        top_sims_idx = np.argpartition(sims[i,:], -top_k)[-top_k:]
        sims_with_idx = [(doc_ids[idx], s) for idx, s in zip(top_sims_idx, sims[i,:][top_sims_idx]) 
                        if s != -np.inf]
        sims_with_idx = sorted(sims_with_idx, key=lambda x:x[1], reverse=True)
        for neg_doc_id, s in sims_with_idx:
            triplets.append(((q_id, pos_doc_id, neg_doc_id), s))

In [13]:
def get_top_k_similar_brute_force(relevant_doc_vector, doc_vecs, relevant_doc_ids, top_k=10, upper_bound=1.0, lower_bound=0.70):
    # calculate similarity
    sim = cosine_similarity(relevant_doc_vector, doc_vecs)
    # set all known relevant documents to -inf
    relevant_doc_idx = [doc_ids_inv[doc_id] for doc_id in relevant_doc_ids if doc_id in doc_ids_inv]
    sim[0,relevant_doc_idx] = -np.inf
    # set all documents outside the boundaries to -inf
    sim = np.where(np.logical_and(sim <= upper_bound, sim >= lower_bound), sim, -np.inf)
    # get the top k document ids with the highest similarity, filtering out -inf
    top_sims_idx = np.argpartition(sim[0,:], -top_k)[-top_k:]
    sims_with_idx = [(doc_ids[idx], s) for idx, s in zip(top_sims_idx, sim[0,:][top_sims_idx]) 
                     if s != -np.inf]
    return sims_with_idx

In [14]:
triplets = []
for q_id, relevant_docs in tqdm(relevance_judgements.items()):
    for pos_doc_id in relevant_docs:
        if pos_doc_id not in doc_ids_inv:
            continue
        relevant_doc_vector = x[doc_ids_inv[pos_doc_id]]
        sims_with_idx = get_top_k_similar_brute_force(relevant_doc_vector=relevant_doc_vector, doc_vecs=x, relevant_doc_ids=relevant_docs, lower_bound=0.40)
        sims_with_idx = sorted(sims_with_idx, key=lambda x:x[1], reverse=True)
        for neg_doc_id, s in sims_with_idx:
            triplets.append(((q_id, pos_doc_id, neg_doc_id), s))

100%|██████████| 367013/367013 [00:34<00:00, 10568.03it/s]


In [15]:
print(len(triplets))
triplets[:10]

1871


[(('2147', 'D2535727', 'D2727364'), 0.51060927),
 (('2147', 'D2535727', 'D1246698'), 0.42265898),
 (('3800', 'D2535659', 'D2998059'), 0.44727397),
 (('3800', 'D2535659', 'D383958'), 0.43441007),
 (('3813', 'D8292', 'D2075844'), 0.47311372),
 (('7389', 'D1584330', 'D392486'), 0.5405516),
 (('7389', 'D1584330', 'D3267610'), 0.4602831),
 (('10896', 'D1516301', 'D1820079'), 0.65886813),
 (('10896', 'D1516301', 'D2264553'), 0.5505372),
 (('11091', 'D469648', 'D250947'), 0.6178612)]

In [24]:
def index_sparse_vectors(x):
    # create an inverse index for the sparse matrix x
    inverted_index = {}
    for doc_idx in tqdm(range(x.shape[0])):
        _, token_ids, token_scores = scipy.sparse.find(x[doc_idx,:])
        for token_id, token_score in zip(token_ids, token_scores):
            if token_id not in inverted_index:
                inverted_index[token_id] = []
            inverted_index[token_id].append((doc_idx, token_score))
    return inverted_index


# def dot_product_sparse(relevant_doc_vector, inverted_index):
#     # if vectors are unit length, it is also the cosine similarity
#     matching_docs = scipy.sparse.lil_array((1,num_docs), dtype='float32')
#     _, token_ids, token_scores = scipy.sparse.find(relevant_doc_vector)
#     for token_id, token_score in zip(token_ids, token_scores):
#         for doc_idx, score in inverted_index[token_id]:
#             matching_docs[0,doc_idx] += score*token_score
#     return matching_docs

def dot_product_sparse(relevant_doc_vector, inverted_index):
    # if vectors are unit length, it is also the cosine similarity
    matching_docs = {}
    t0 = time.time()
    _, token_ids, token_scores = scipy.sparse.find(relevant_doc_vector)
    t1 = time.time()
    for token_id, token_score in zip(token_ids, token_scores):
        if len(inverted_index[token_id]) > 10000:
            print(len(inverted_index[token_id]))
        for doc_idx, score in inverted_index[token_id]:
            if doc_idx not in matching_docs:
                matching_docs[doc_idx] = 0
            #matching_docs[doc_idx] += score*token_score
    t2 = time.time()
    print(t1-t0, t2-t1)
    return matching_docs


def get_top_k_similar(relevant_doc_vector, inverted_index, relevant_doc_ids, top_k=10, upper_bound=1.0, lower_bound=0.70):
    global sparse_time
    # calculate similarity
    t1 = time.time()
    sim = dot_product_sparse(relevant_doc_vector, inverted_index)
    sparse_time += time.time() - t1
    return
    # set all known relevant documents to -inf
    relevant_doc_idx = [doc_ids_inv[doc_id] for doc_id in relevant_doc_ids if doc_id in doc_ids_inv]
    sim[0,relevant_doc_idx] = 0.0
    # set all documents outside the boundaries to -inf
    sim = np.where(np.logical_and(sim <= upper_bound, sim >= lower_bound), sim, 0.0)
    # get the top k document ids with the highest similarity, filtering out -inf
    top_sims_idx = np.argpartition(sim[0,:], -top_k)[-top_k:]
    sims_with_idx = [(doc_ids[idx], s) for idx, s in zip(top_sims_idx, sim[0,:][top_sims_idx]) 
                     if s != -np.inf]
    return sims_with_idx

In [9]:
inverted_index = index_sparse_vectors(x)

100%|██████████| 99623/99623 [00:26<00:00, 3780.84it/s]


In [25]:
triplets = []
sparse_time = 0
for q_id, relevant_docs in tqdm(relevance_judgements.items()):
    for pos_doc_id in relevant_docs:
        if pos_doc_id not in doc_ids_inv:
            continue
        relevant_doc_vector = x[doc_ids_inv[pos_doc_id]]
        sims = dot_product_sparse(relevant_doc_vector, inverted_index)
        #print(len(sims))

print(sparse_time)

  0%|          | 0/367013 [00:00<?, ?it/s]

10584
10685
10919
15553
17740
24811
21745
18898
22190
15051
11417
13841
18623
17295
14873
20623
26262
18466


  0%|          | 34/367013 [00:00<1:14:01, 82.63it/s]

10363
11089
13424
37708
16466
42770
10799
23381
27262
39636
13848
10111
33593
16272
24285
39384
22691
0.0003273487091064453 0.4091305732727051
26103
22662
17849
24811
41426
21745
37226
29086
18745
12841
14271
10976
11640
12975
12733
26975
24062
15634
18801
11547
20470
28186
38956
12600
15363
20691
38428
14861
10298
26330
11922
24382
10151
28849
26503
47210
25175
43058
12414
18148
14392
13554
12755
11089
18058
18777
45674
25840
42770
43946
12073
22731
17833
41385
32291
44592
36880
12184
12029
18140
14456
23495
11091
18608
15527
13713
21801
43341
45679
16931
15803
28863
12396
43815
32888
47640
43450
25386
23240
31118
46463
37585
48515
39384
24455
14068
23825
45298
11348
29513


  0%|          | 68/367013 [00:01<2:22:32, 42.90it/s]

...

  0%|          | 87/367013 [00:02<4:17:54, 23.71it/s]

10077
37585
12256
34530
45298
28932
35864
29513
33095
38418
22805
10264
27469
22691
28342
32243
0.00039649009704589844 0.9874193668365479
26103
17873
14553
16995
12599
36965
14263
14544
17740
24811
41426
24117
14782
37226
30647
29903
10625
27095
22190
15232
13776
19212
11454
24856
26975
18224
24848
26116
14647
11340
18801
11417
13841
29641
14257
17295
38956
20062
15363
27765
11255
14716
14861
26959
20556
26330
24382
23107
16231
26503
12270
24498
38153
26262
21156
14753
15311
14111
22807
43058
12414
18148
13554
26738
10183
37708
11190
47790
18777
13204
29448
21439
13790
11397
16095
42770
12838
43946
23455
10867
23381
17833
18575
41385
18444
34011
11087
10708
11816
11374


  0%|          | 161/367013 [00:03<2:31:29, 40.36it/s]

13167
14563
14430
13803
12373
16106
18540
18608
18965
21801
45679
13586


KeyboardInterrupt: 

In [23]:
triplets = []
sparse_time = 0
i = 0
for q_id, relevant_docs in tqdm(relevance_judgements.items()):
    for pos_doc_id in relevant_docs:
        if pos_doc_id not in doc_ids_inv:
            continue
        relevant_doc_vector = x[doc_ids_inv[pos_doc_id]]
        sims_with_idx = get_top_k_similar(relevant_doc_vector=relevant_doc_vector, inverted_index=inverted_index, relevant_doc_ids=relevant_docs, lower_bound=0.40)
        continue
        sims_with_idx = sorted(sims_with_idx, key=lambda x:x[1], reverse=True)
        for neg_doc_id, s in sims_with_idx:
            triplets.append(((q_id, pos_doc_id, neg_doc_id), s))
    i += 1
    if i == 10000:
        break

print(sparse_time)

  3%|▎         | 9999/367013 [00:06<04:00, 1483.46it/s]

6.696434736251831


In [16]:
print(len(triplets))
triplets[:3]

1872

In [18]:
skipped = 0
for q_id, relevant_docs in relevance_judgements.items():
    sims_with_idx = None
    for rel_doc_id in relevant_docs:
        if rel_doc_id not in doc_ids_inv:
            skipped += 1
            continue
print("skipped", skipped)

skipped 365877


In [12]:
def get_documents():
    documents = {}
    with open(document_file, "r") as fp:
        for doc_line in fp:
            data = doc_line.strip().split(sep="\t")
            if len(data) != 4:
                #print(len(data), data)
                continue
            doc_id, doc_url, doc_title, doc_text = data
            documents[doc_id] = doc_text
    return documents

In [13]:
documents = get_documents()
queries = read_queries()

In [14]:
print(queries["2147"])
print("===== POS =====")
print([documents['D2535727']])
print("===== NEG =====")
print([documents['D2727364']])
print([documents['D1246698']])

average speed is defined as
===== POS =====
['Velocity The average speed of an object is defined as the distance traveled divided by the time elapsed. Velocity is a vector quantity, and average velocity can be defined as the displacement divided by the time. For the special case of straight line motion in the x direction, the average velocity takes the form: The units for velocity can be implied from the definition to be meters/second or in general any distance unit over any time unit. You can approach an expression for the instantaneous velocity at any point on the path by taking the limit as the time interval gets smaller and smaller. Such a limiting process is called a derivative and the instantaneous velocity can be defined as Average velocity, straight line motion Average velocity, general case Index Motion concepts Hyper Physics ***** Mechanics R Nave Go Back']
===== NEG =====
['Kinematics with Graphs Since you are not allowed to use calculators, SAT II Physics places a heavy emp

In [19]:
print(queries["10896"])
print("===== POS =====")
print([documents['D1516301']])
print("===== NEG =====")
print([documents['D1820079']])
print([documents['D2264553']])

actor who played the original chewbacca
===== POS =====
['"Peter Mayhew From Wikipedia, the free encyclopedianavigation search For other people named Peter Mayhew, see Peter Mayhew (disambiguation). This article\'s lead section does not adequately summarize key points of its contents. Please consider expanding the lead to provide an accessible overview of all important aspects of the article. Please discuss this issue on the article\'s talk page. (January 2015)Peter Mayhew Mayhew at the Florida Super Con, June 2015Born 19 May 1944 (age 73)Barnes, London, England Residence Boyd, Texas, USNationality British, American Occupation Actor Years active 1976–present Known for Playing Chewbacca in the Star Wars franchise Height 7 ft 3 in (221 cm)Spouse (s) Angie Luker (1999–present)Peter Mayhew (born 19 May 1944) [1] is an English-American actor notable for portraying Chewbacca in the Star Wars film series. Contents [ hide ]1 Early life2 Career2.1 Early work2.2 Chewbacca2.3 Other work2.4 Books3